In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_3_5

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
SYSTEM_PROMPT="""
I want you to act as a software engineer.
Your task is to read this following user story and extract all functions mentioned in the user story.
You must put all the sentences related to that function into their corresponding title.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Function 1":[
    "functionName": "Clear name of the function",
    "objective": "Hoal of the function",
    "Descriptive sentences related to the function": [
      "Sentence 1",
      "Sentence 2",
      "Sentence 3"
    ],
  ],
}
"""

In [3]:
user_prompt="""
The Library Management System will be PC-base with a internet, allowing library users to search for books,seminars and library staff members to manage the book inventory and user database.
DLS SYSTEM is used for Library Manager, Librarian, and Library User.
However, it is possible to exchange data with other system through external interface if required.
This system is Web based, there will be a need to provide PC Server hardware connected to the internet.
Therefore, the system should be designed for easy to use, providing help instructions, and appropriate error messages for invalid user inputs.
Library user is allowed to use the DLSSYSTEM only for searching book records.
The DLSSYSTEM should not have any unscheduled down time during library operation hours.
Any down time in operation hours has significant impact to the operation and cause inconvenience to everyone in library.
As such, all interfaces should provide easy access to help as well as clearly indicate the current state of the user’s transaction when the user isn’t idle.
Because of the complexity of the data model, Library Staff will need to be able to edit multiple records simultaneously and create links between them.
For example, after a record has been edited, but before it has be en “saved” into the repository two versions of the record exist.
Within the system, logging will be used to provide a trail of transactions that have taken place.
SRS-008: When download the books, the system shall display the information of the e-book which is just being downloaded including: ISBN, title, location.
SRS-011: Whenever the "date" data is needed, it shall be entered only by choose date from a online calendar.
The system shall allow the user choose language option which the SRS-019: If the search result are a list of books, the system shall allow the user to choose any one of them to see the details.
The Patron information report shall be generated by users who have librarian account.
The book purchase report shall only be generated by managers or users with defined privileges.
SRS-7: Database update data shall be committed to the database only after the managers have approved.
The system shall be recovered within 10 minutes if it is down.
The system shall be recovered without intervention at user terminal if it is down.
The system shall show appropriate messages at terminal when system is down.
The system shall have 99% reliability during library operating hours.
SRS-012: Scheduled down time after library operating hours shall not be more than 1 hour per day.
The system shall generate error messages when the user attempts to enter invalid data.
Instructors will be using the system in collaboration with Concourse to provide access to course materials and content stored within the DML.
For example, after a record has been edited, but before it has be en “saved” into main database .
The RDB administrator, who will be able to select an existing word or add a new one (hence it is a less controlled list than the Subjects, suggests keywords.
"""

In [4]:
promptTestCase = [
{ "role": "system", "content": SYSTEM_PROMPT},
{ "role": "user", "content": user_prompt}
]
gpt_response = askJSON(promptTestCase, client, model)


In [5]:
print(gpt_response)

{
  "Library Management System Functions": [
    {
      "functionName": "Search for Books, Seminars, and Library Staff Members",
      "objective": "Allow library users to search for books, seminars, and library staff members",
      "Descriptive sentences related to the function": [
        "The Library Management System will be PC-based with internet, allowing library users to search for books, seminars, and library staff members",
        "Library user is allowed to use the DLSSYSTEM only for searching book records"
      ]
    },
    {
      "functionName": "Manage Book Inventory and User Database",
      "objective": "Allow library staff members to manage the book inventory and user database",
      "Descriptive sentences related to the function": [
        "Library staff members to manage the book inventory and user database"
      ]
    },
    {
      "functionName": "Exchange Data with Other Systems",
      "objective": "Exchange data with other systems through external interf

In [10]:
def write_json_to_txt(json_data, file_name):
    def process_item(item, file, indent=0):
        indent_str = ' ' * indent
        if isinstance(item, dict):
            for key, value in item.items():
                file.write(f"{indent_str}{key}:\n")
                process_item(value, file, indent + 2)
        elif isinstance(item, list):
            for value in item:
                file.write(f"{indent_str}- ")
                if isinstance(value, (dict, list)):
                    file.write("\n")
                    process_item(value, file, indent + 2)
                else:
                    file.write(f"{value}\n")
        else:
            file.write(f"{indent_str}{item}\n")
    
    with open(file_name, 'w') as file:
        process_item(json_data, file)

In [7]:
def write_json_to_txt_raw2(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Application:  {test_case}\n")
            for json_usecase in details:
                for key, value in json_usecase.items():
                    file.write(f"{key} : {value} \n")
            file.write("\n")

In [8]:
def write_to_txt_file(data, file_name):
    formatted_json = json.dumps(data, indent=4)
    with open(file_name, 'w') as file:
        file.write(formatted_json)

In [13]:
json_data = json.loads(gpt_response)
write_json_to_txt_raw2((json_data), "function.txt")
write_to_txt_file((json_data), "function1.txt")
write_json_to_txt(json_data, "function.txt")